## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-05-46-00 +0000,bbc,Aviation authority says Boeing fuel switches a...,https://www.bbc.com/news/articles/ce9xpgnx3vdo
1,2025-07-14-05-26-09 +0000,nypost,Biden defends controversial autopen use for ma...,https://nypost.com/2025/07/14/us-news/biden-de...
2,2025-07-14-05-14-12 +0000,bbc,Public transport 'too costly' for young people,https://www.bbc.com/news/articles/cyvj5jj9v16o
3,2025-07-14-05-07-22 +0000,nypost,UC Berkeley professor killed by masked gunman ...,https://nypost.com/2025/07/14/us-news/uc-berke...
4,2025-07-14-05-00-11 +0000,nyt,Monday Briefing: America’s Brain Drain and the...,https://www.nytimes.com/2025/07/14/briefing/ha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
156,trump,9
251,mamdani,5
201,nyc,4
254,against,4
180,he,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,32
17,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...,30
24,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...,28
37,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...,27
48,2025-07-13-12-58-08 +0000,nypost,President Trump reveals what he really wants h...,https://nypost.com/2025/07/13/us-news/presiden...,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,32,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
24,28,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...
17,21,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
27,20,2025-07-13-21-33-00 +0000,nypost,Eric Adams rips Andrew Cuomo’s planned bid to ...,https://nypost.com/2025/07/13/us-news/adams-ch...
19,18,2025-07-13-23-10-41 +0000,nypost,Devoted dad gunned down en route to pick up da...,https://nypost.com/2025/07/13/us-news/nyc-dad-...
37,18,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
6,17,2025-07-14-04-47-53 +0000,nypost,Teddy bear purportedly wrapped in human skin l...,https://nypost.com/2025/07/14/us-news/teddy-be...
5,16,2025-07-14-04-55-54 +0000,nypost,Elmo’s X account goes rogue as antisemitic hac...,https://nypost.com/2025/07/14/us-news/elmos-x-...
23,15,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...
12,13,2025-07-14-02-35-50 +0000,nypost,Arizona police officers deliver pizza to shock...,https://nypost.com/2025/07/13/us-news/arizona-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
